In [1]:
# Final Project physics 140
# Created by Peter Nguyen 
# Prompt: Use reinforced learning and the bellman equation to train an agent to solve mazes.

In [2]:
# module imports
import numpy as np
import matplotlib.pyplot as plt
import random

In [3]:
# Creation of the graph

# Width of Graph
W = 3
# Length of Graph
L = 3
# edge
Edge = 0

# maze initializing
maze = np.zeros((W,L))
print(maze)
#maze[0,1] = 1
#maze[0,0] = 2

[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]


In [4]:
# define class of agent
class agent():
    # Default Constructor
    def __init__(self, name, pos_x = 0, pos_y = 0):
        self.name = name
        self.position_x = pos_x
        self.position_y = pos_y
        print("agent created:",name)
        
    def curr_position(self):
        return (self.position_x,self.position_y)
        
    # Movement method for our agent
    def movement(self, maze, move):
        view_maze(maze)
        print("new movement made")
        # go down
        if move == 1:
            self.position_y+=1
            print("right")
        # go up
        elif move == 2:
            self.position_y-=1
            print("left")
        # move right
        elif move == 3:
            self.position_x+=1
            print("down")
        # move left
        elif move == 4:
            self.position_x-=1
            print("up")
            
            

In [5]:
# Test Agent
a1 = agent("Peter",1,1)

agent created: Peter


In [6]:
def view_maze(maze):
    print("   1  2  3")
    count = 0
    for i in maze:
        count+=1
        print(count,i)

In [7]:
# boundary checker + movement functionality

#start
print("start")
x_pos, y_pos = a1.curr_position()

for i in range(10):
    print("x",x_pos + 1,"y",y_pos + 1)
    x_pos, y_pos = a1.curr_position()
    move = np.random.randint(1,5)
    # Checks for boundaries in down
    if x_pos < W - 1 and move == 3:
        a1.movement(maze, move)
    # Checks for boundaries in up
    elif x_pos > Edge and move == 4:
        a1.movement(maze, move)
    #checks for boundaries in left
    elif y_pos < L - 1 and move == 1:
        a1.movement(maze, move)
    # Checks for boundaries in right
    elif y_pos > Edge and move == 2:
        a1.movement(maze, move)
    # indicate that the agent hit the boundary
    else:
        if move == 1:
            print("boundary hit on right")
        elif move == 2:
            print("boundary hit on left")
        elif move == 3:
            print("boundary hit on down")
        elif move == 4:
            print("boundary hit on up")
    maze[x_pos,y_pos] = i + 1
    view_maze(maze)
    print("x",x_pos + 1,"y",y_pos + 1)
    print('---------- END OF TURN ---------------')

start
x 2 y 2
   1  2  3
1 [0. 0. 0.]
2 [0. 0. 0.]
3 [0. 0. 0.]
new movement made
left
   1  2  3
1 [0. 0. 0.]
2 [0. 1. 0.]
3 [0. 0. 0.]
x 2 y 2
---------- END OF TURN ---------------
x 2 y 2
boundary hit on left
   1  2  3
1 [0. 0. 0.]
2 [2. 1. 0.]
3 [0. 0. 0.]
x 2 y 1
---------- END OF TURN ---------------
x 2 y 1
   1  2  3
1 [0. 0. 0.]
2 [2. 1. 0.]
3 [0. 0. 0.]
new movement made
down
   1  2  3
1 [0. 0. 0.]
2 [3. 1. 0.]
3 [0. 0. 0.]
x 2 y 1
---------- END OF TURN ---------------
x 2 y 1
   1  2  3
1 [0. 0. 0.]
2 [3. 1. 0.]
3 [0. 0. 0.]
new movement made
right
   1  2  3
1 [0. 0. 0.]
2 [3. 1. 0.]
3 [4. 0. 0.]
x 3 y 1
---------- END OF TURN ---------------
x 3 y 1
boundary hit on down
   1  2  3
1 [0. 0. 0.]
2 [3. 1. 0.]
3 [4. 5. 0.]
x 3 y 2
---------- END OF TURN ---------------
x 3 y 2
   1  2  3
1 [0. 0. 0.]
2 [3. 1. 0.]
3 [4. 5. 0.]
new movement made
up
   1  2  3
1 [0. 0. 0.]
2 [3. 1. 0.]
3 [4. 6. 0.]
x 3 y 2
---------- END OF TURN ---------------
x 3 y 2
   1  2  3
1 [0. 0. 0.]

In [8]:
# logic for running
# 1s are walls
# 2s are end goal
# 0s are path